In [1]:
import numpy
import pandas
import torch
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df = pandas.read_json("datasets/joke_info.json")
df

,joke_id,USF1000_rating,USF1001_rating,USF1002_rating,datasource_joke_id,jokeText,jester,reddit,stupidstuff
0,RE5tz52q,1,8,0,5tz52q,I hate how you cant even say black paint anymo...,0,1,0
1,RE5tz4dd,1,8,0,5tz4dd,What's the difference between a Jew in Nazi Ge...,0,1,0
2,RE5tz319,0,9,8,5tz319,I recently went to America....\n...and being t...,0,1,0
3,RE5tz2wj,3,9,0,5tz2wj,"Brian raises his hand and says, “He’s in Heave...",0,1,0
4,RE5tz1pc,5,3,8,5tz1pc,You hear about the University book store worke...,0,1,0
...,...,...,...,...,...,...,...,...,...
160,SS2771,7,10,0,2771,Q: How many Field Service Engineers does it ta...,0,0,1
161,SS1365,8,10,0,1365,Yo mamma so dumb she sold her car for gasoline...,0,0,1
162,SS570,8,3,0,570,"A woman gets home, schreeches her car into the...",0,0,1
163,SS431,7,7,0,431,The psychology instructor had just finished a ...,0,0,1


In [5]:
USF_users = pandas.read_csv('datasets/users.csv')
USF_users

,user_id,name,age,gender,ethnicity,country,location,avg_user_rating,city,buddy
0,USF1000,Tanya,24,Female,South East Asian,India,San Francisco,5.968586,Delhi,USF1001
1,USF1001,rishab,26,Male,South East Asian,India,San Francisco,6.727749,Mumbai,USF1000
2,USF1002,Parisa,26,Female,Middle Eastern,Iran,San Francisco,0.806283,Ahvaz,USF1001


In [6]:
# add a column `avg_user_rating` to `USF_users` dataframe that contains the average rating of each user across all jokes in the df dataframe
# so for a user with id x in USF_users, the value of `avg_user_rating` should be the average rating of column 'USF100{x}_rating' in df dataframe
# if a user has not rated any joke, the value of `avg_user_rating` should be 0

avg_user_rating = []
for index, row in USF_users.iterrows():
    user_id = row['user_id']
    avg_user_rating.append(df[f'{user_id}_rating'].mean())

USF_users['avg_user_rating'] = avg_user_rating
USF_users

,user_id,name,age,gender,ethnicity,country,location,avg_user_rating,city,buddy
0,USF1000,Tanya,24,Female,South East Asian,India,San Francisco,6.103030,Delhi,USF1001
1,USF1001,rishab,26,Male,South East Asian,India,San Francisco,6.793939,Mumbai,USF1000
2,USF1002,Parisa,26,Female,Middle Eastern,Iran,San Francisco,1.096970,Ahvaz,USF1001


In [7]:
# make a new  dataframe df2 that recalculates the ratings of all jokes by calculating the difference between the average rating of the user and the rating of the joke
# so for a user with id x in USF_users, the value of `x_rating` should be the difference between the average rating of the user from the USF_users dataframe and the rating of the joke from the df dataframe
# if a user has not rated any joke, the value of `x_rating` should be 0

df2 = df.copy()
for index, row in USF_users.iterrows():
    user_id = row['user_id']
    df2[f'{user_id}_rating'] =  row['avg_user_rating'] - df2[f'{user_id}_rating']

df2

,joke_id,USF1000_rating,USF1001_rating,USF1002_rating,datasource_joke_id,jokeText,jester,reddit,stupidstuff
0,RE5tz52q,5.10303,-1.206061,1.09697,5tz52q,I hate how you cant even say black paint anymo...,0,1,0
1,RE5tz4dd,5.10303,-1.206061,1.09697,5tz4dd,What's the difference between a Jew in Nazi Ge...,0,1,0
2,RE5tz319,6.10303,-2.206061,-6.90303,5tz319,I recently went to America....\n...and being t...,0,1,0
3,RE5tz2wj,3.10303,-2.206061,1.09697,5tz2wj,"Brian raises his hand and says, “He’s in Heave...",0,1,0
4,RE5tz1pc,1.10303,3.793939,-6.90303,5tz1pc,You hear about the University book store worke...,0,1,0
...,...,...,...,...,...,...,...,...,...
160,SS2771,-0.89697,-3.206061,1.09697,2771,Q: How many Field Service Engineers does it ta...,0,0,1
161,SS1365,-1.89697,-3.206061,1.09697,1365,Yo mamma so dumb she sold her car for gasoline...,0,0,1
162,SS570,-1.89697,3.793939,1.09697,570,"A woman gets home, schreeches her car into the...",0,0,1
163,SS431,-0.89697,-0.206061,1.09697,431,The psychology instructor had just finished a ...,0,0,1


In [8]:
# create a new  matrix dataframe `similarity_matrix` that contains the similarity between each pair of users
# the similarity between two users is the division of (sum of the products of each rating of the two users) by (product of the square root of the sum of the squares of the ratings of each user)
# so for a user with id x in USF_users, the value of `x` in `similarity_matrix` should be the similarity between the user of that row and the `x` user

similarity_matrix = pandas.DataFrame(index=USF_users['user_id'], columns=USF_users['user_id'])

for index, row in USF_users.iterrows():
    user_id = row['user_id']
    for index2, row2 in USF_users.iterrows():
        user_id2 = row2['user_id']
        similarity_matrix[user_id][user_id2] = (df2[f'{user_id}_rating'] * df2[f'{user_id2}_rating']).sum() / (numpy.sqrt((df2[f'{user_id}_rating']**2).sum()) * numpy.sqrt((df2[f'{user_id2}_rating']**2).sum()))

similarity_matrix




user_id,USF1000,USF1001,USF1002
user_id,,,
USF1000,1.0,0.257828,-0.109062
USF1001,0.257828,1.0,0.05376
USF1002,-0.109062,0.05376,1.0


In [11]:
USF1002_ratings = pandas.read_csv('datasets/ratings_USF1002.csv')
USF1002_ratings = USF1002_ratings[USF1002_ratings['joke_id'].str.startswith('JE')]

USF1000_ratings = pandas.read_csv('datasets/ratings_USF1000.csv')
USF1000_ratings = USF1000_ratings[USF1000_ratings['joke_id'].str.startswith('JE')]

USF1001_ratings = pandas.read_csv('datasets/ratings_USF1001.csv')
USF1001_ratings = USF1001_ratings[USF1001_ratings['joke_id'].str.startswith('JE')]

In [12]:
pred = pandas.DataFrame(columns=['joke_id', 'USF1002_rating', 'USF1000_rating', 'USF1001_rating'])
pred['joke_id'] = USF1002_ratings['joke_id']
pred['USF1002_rating'] = USF1002_ratings['rating']
pred['USF1000_rating'] = USF1000_ratings['rating']
pred['USF1001_rating'] = USF1001_ratings['rating']
pred

,joke_id,USF1002_rating,USF1000_rating,USF1001_rating
0,JE1,0,NaN,NaN
1,JE2,0,NaN,NaN
2,JE3,8,NaN,NaN
3,JE4,0,NaN,NaN
4,JE5,8,NaN,8.0
5,JE6,0,NaN,8.0
6,JE7,0,NaN,9.0
7,JE8,9,NaN,9.0
8,JE9,0,NaN,3.0
9,JE10,3,NaN,4.0


In [13]:
extracted = pred.copy()
extracted = extracted.dropna()
extracted

,joke_id,USF1002_rating,USF1000_rating,USF1001_rating
39,JE40,1,1.0,9.0
40,JE41,0,1.0,5.0
41,JE42,0,0.0,10.0
42,JE43,0,3.0,0.0
43,JE44,0,5.0,10.0
44,JE45,0,4.0,10.0
45,JE46,0,0.0,10.0
46,JE47,0,2.0,10.0
47,JE48,3,3.0,10.0
48,JE49,0,0.0,8.0


In [14]:
USF_users[USF_users['user_id'] == 'USF1000'].columns

Index(['user_id', 'name', 'age', 'gender', 'ethnicity', 'country', 'location',
       'avg_user_rating', 'city', 'buddy'],
      dtype='object')

In [15]:
# define a function 'predict_rating' that takes as input a joke_id and a user_id and returns the predicted rating of the joke by the user
# for user `x` in `USF_users`, the predicted rating of a joke `y` by the user `x` is the 
# sum of (average rating of user `x`) and (sum of the product of the similarity between `x` and each user `z` and the rating of the joke `y` by the user `z` divided by the sum of absolute similarities between `x` and each user `z`)

def predict_rating(joke_id, user_id):
#     print(joke_id)
    user_avg = USF_users[USF_users['user_id'] == user_id]['avg_user_rating'].values[0]
    print(user_avg)
    similarity_sum = 0
    similarity_rating_sum = 0
    for index, row in USF_users.iterrows():
        user_id2 = row['user_id']
        if user_id2 != user_id:
            userid2_rating = USF_users[USF_users['user_id'] == user_id2]['avg_user_rating'].values[0] - extracted[extracted['joke_id']==joke_id][f'{user_id2}_rating'].values[0]
            similarity_sum += similarity_matrix[user_id][user_id2]
            similarity_rating_sum += similarity_matrix[user_id][user_id2] * userid2_rating
            print(f"user_id2: {user_id2} \t {similarity_rating_sum} \t {similarity_sum}")
    return round(user_avg + (similarity_rating_sum / similarity_sum))

In [16]:
extracted['USF1002_predicted_rating'] = extracted.apply(lambda row: predict_rating(row['joke_id'], 'USF1002'), axis=1)
extracted

1.096969696969697
user_id2: USF1000 	 -0.55654657955779 	 -0.10906197817937689
user_id2: USF1001 	 -0.6751433719048914 	 -0.05530244318687213
1.096969696969697
user_id2: USF1000 	 -0.55654657955779 	 -0.10906197817937689
user_id2: USF1001 	 -0.46010523193487235 	 -0.05530244318687213
1.096969696969697
user_id2: USF1000 	 -0.6656085577371669 	 -0.10906197817937689
user_id2: USF1001 	 -0.837964885076773 	 -0.05530244318687213
1.096969696969697
user_id2: USF1000 	 -0.3384226231990362 	 -0.10906197817937689
user_id2: USF1001 	 0.026816399386405232 	 -0.05530244318687213
1.096969696969697
user_id2: USF1000 	 -0.12029866684028244 	 -0.10906197817937689
user_id2: USF1001 	 -0.2926549941798886 	 -0.05530244318687213
1.096969696969697
user_id2: USF1000 	 -0.22936064501965933 	 -0.10906197817937689
user_id2: USF1001 	 -0.4017169723592655 	 -0.05530244318687213
1.096969696969697
user_id2: USF1000 	 -0.6656085577371669 	 -0.10906197817937689
user_id2: USF1001 	 -0.837964885076773 	 -0.055302443186

,joke_id,USF1002_rating,USF1000_rating,USF1001_rating,USF1002_predicted_rating
39,JE40,1,1.0,9.0,13
40,JE41,0,1.0,5.0,9
41,JE42,0,0.0,10.0,16
42,JE43,0,3.0,0.0,1
43,JE44,0,5.0,10.0,6
44,JE45,0,4.0,10.0,8
45,JE46,0,0.0,10.0,16
46,JE47,0,2.0,10.0,12
47,JE48,3,3.0,10.0,10
48,JE49,0,0.0,8.0,14


In [17]:
user_id2= 'USF1000'
# extracted['joke_id']=='JE40'
extracted[extracted['joke_id']=='JE40'][f'{user_id2}_rating'].values[0]

1.0

In [18]:
finals = pandas.read_json('../datasets/final_user_joke_dataset.json')
finals

,joke_id,USF1000_rating,USF1001_rating,USF1002_rating,datasource_joke_id,jokeText,jester,reddit,stupidstuff,toxic,...,ethnicity_Middle Eastern,ethnicity_South East Asian,country_India,country_Iran,location_San Francisco,city_Ahvaz,city_Delhi,city_Mumbai,buddy_USF1000,buddy_USF1001
0,RE5tz52q,1,8,0,5tz52q,I hate how you cant even say black paint anymo...,0,1,0,0.948757,...,0.000000,0.300000,0.300000,0.000000,0.300000,0.000000,0.033333,0.266667,0.266667,0.033333
1,RE5tz4dd,1,8,0,5tz4dd,What's the difference between a Jew in Nazi Ge...,0,1,0,0.740485,...,0.000000,0.300000,0.300000,0.000000,0.300000,0.000000,0.033333,0.266667,0.266667,0.033333
2,RE5tz319,0,9,8,5tz319,I recently went to America....\n...and being t...,0,1,0,0.713800,...,0.266667,0.300000,0.300000,0.266667,0.566667,0.266667,0.000000,0.300000,0.300000,0.266667
3,RE5tz2wj,3,9,0,5tz2wj,"Brian raises his hand and says, “He’s in Heave...",0,1,0,0.863763,...,0.000000,0.400000,0.400000,0.000000,0.400000,0.000000,0.100000,0.300000,0.300000,0.100000
4,RE5tz1pc,5,3,8,5tz1pc,You hear about the University book store worke...,0,1,0,0.556188,...,0.266667,0.266667,0.266667,0.266667,0.533333,0.266667,0.166667,0.100000,0.100000,0.433333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,SS2771,7,10,0,2771,Q: How many Field Service Engineers does it ta...,0,0,1,0.647737,...,0.000000,0.566667,0.566667,0.000000,0.566667,0.000000,0.233333,0.333333,0.333333,0.233333
161,SS1365,8,10,0,1365,Yo mamma so dumb she sold her car for gasoline...,0,0,1,0.736433,...,0.000000,0.600000,0.600000,0.000000,0.600000,0.000000,0.266667,0.333333,0.333333,0.266667
162,SS570,8,3,0,570,"A woman gets home, schreeches her car into the...",0,0,1,0.807468,...,0.000000,0.366667,0.366667,0.000000,0.366667,0.000000,0.266667,0.100000,0.100000,0.266667
163,SS431,7,7,0,431,The psychology instructor had just finished a ...,0,0,1,0.673749,...,0.000000,0.466667,0.466667,0.000000,0.466667,0.000000,0.233333,0.233333,0.233333,0.233333


In [19]:
USF_users

,user_id,name,age,gender,ethnicity,country,location,avg_user_rating,city,buddy
0,USF1000,Tanya,24,Female,South East Asian,India,San Francisco,6.103030,Delhi,USF1001
1,USF1001,rishab,26,Male,South East Asian,India,San Francisco,6.793939,Mumbai,USF1000
2,USF1002,Parisa,26,Female,Middle Eastern,Iran,San Francisco,1.096970,Ahvaz,USF1001


In [21]:
# for idx, row in USF_users.iterrows():
#     print(idx, row)
#     USF_users.loc[idx]['city'] = 'a'
# USF_users[USF_users['user_id']=='USF1000']['city'] = 'Delhi'
# USF_users.loc[0, 'city'] = 'Delhi'
# USF_users.loc[1, 'city'] = 'Mumbai'
# USF_users.loc[2, 'city'] = 'Ahvaz'
# USF_users = USF_users.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
# USF_users

In [22]:
USF_users.to_csv('../datasets/users.csv', index=False)
USF_users = pandas.read_csv('../datasets/users.csv')
USF_users

,user_id,name,age,gender,ethnicity,country,location,avg_user_rating,city,buddy
0,USF1000,Tanya,24,Female,South East Asian,India,San Francisco,6.103030,Delhi,USF1001
1,USF1001,rishab,26,Male,South East Asian,India,San Francisco,6.793939,Mumbai,USF1000
2,USF1002,Parisa,26,Female,Middle Eastern,Iran,San Francisco,1.096970,Ahvaz,USF1001


In [23]:
# in USF_users  create a new column buddy which contains the user_id of the user who has the highest similarity with the user after excluding the user itself
# if there are multiple users with the same highest similarity, choose the first one

USF_users['buddy'] = ''
for index, row in USF_users.iterrows():
    user_id = row['user_id']
    max_similarity = 0
    buddy = ''
    for index2, row2 in USF_users.iterrows():
        user_id2 = row2['user_id']
        if user_id2 != user_id and similarity_matrix[user_id][user_id2] > max_similarity:
            max_similarity = similarity_matrix[user_id][user_id2]
            buddy = user_id2
    USF_users.loc[index, 'buddy'] = buddy

USF_users

,user_id,name,age,gender,ethnicity,country,location,avg_user_rating,city,buddy
0,USF1000,Tanya,24,Female,South East Asian,India,San Francisco,6.103030,Delhi,USF1001
1,USF1001,rishab,26,Male,South East Asian,India,San Francisco,6.793939,Mumbai,USF1000
2,USF1002,Parisa,26,Female,Middle Eastern,Iran,San Francisco,1.096970,Ahvaz,USF1001


In [24]:
USF_users.to_csv('../datasets/users.csv', index=False)
USF_users = pandas.read_csv('../datasets/users.csv')
USF_users

,user_id,name,age,gender,ethnicity,country,location,avg_user_rating,city,buddy
0,USF1000,Tanya,24,Female,South East Asian,India,San Francisco,6.103030,Delhi,USF1001
1,USF1001,rishab,26,Male,South East Asian,India,San Francisco,6.793939,Mumbai,USF1000
2,USF1002,Parisa,26,Female,Middle Eastern,Iran,San Francisco,1.096970,Ahvaz,USF1001
